In [1]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [2]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1735912169

In [3]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-03 15:49:29,497 INFO: Initializing external client
2025-01-03 15:49:29,498 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-03 15:49:31,183 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [4]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732456169.0

In [5]:
today, starting_timestamp

(1735912169, 1732456169.0)

In [6]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732492800,252.93,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267
1,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
2,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
3,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987
4,1732838400,237.59,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213


In [7]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


,timestamp,open
0,1732492800,97994.1
1,1732579200,93000.0
2,1732665600,91914.5
3,1732752000,95952.9
4,1732838400,95637.8


In [8]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=1
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['value_classification'] = fng_batch_data['value_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 


,value,value_classification,timestamp
0,82,4,1.732493e+09
1,79,4,1.732579e+09
2,75,3,1.732666e+09
3,77,4,1.732752e+09
4,78,4,1.732838e+09


In [9]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=3,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/rh/xp6gtwpn19s_8lqdxq8sdl0r0000gn/T/4a023e16-8d68-4968-be6d-df329339d98c/solana_price_daily_model_10d/3


<Sequential name=sequential, built=True>

In [10]:
print(solana_batch_data['timestamp'].dtype)
print(bitcoin_batch_data['timestamp'].dtype)
print(fng_batch_data['timestamp'].dtype)

print(solana_batch_data['timestamp'].head())
print(bitcoin_batch_data['timestamp'].head())
print(fng_batch_data['timestamp'].head())


int64
int64
float64
0    1732492800
1    1732579200
2    1732665600
3    1732752000
4    1732838400
Name: timestamp, dtype: int64
0    1732492800
1    1732579200
2    1732665600
3    1732752000
4    1732838400
Name: timestamp, dtype: int64
0    1.732493e+09
1    1.732579e+09
2    1.732666e+09
3    1.732752e+09
4    1.732838e+09
Name: timestamp, dtype: float64


In [11]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,value,value_classification,open_solana
0,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267,97994.1,82,4,252.93
1,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
2,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
3,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26
4,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213,95637.8,78,4,237.59


In [12]:
scaler_labels = joblib.load('scaler_labels.pkl')
scaler_features = joblib.load('scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [13]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'value', 'value_classification', 'open_solana'],
      dtype='object')

In [14]:
predictions = model.predict(X)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [15]:
predictions

array([0.435249  , 0.5247553 , 0.4310208 , 0.3410556 , 0.37024307,
       0.5414145 , 0.5247258 , 0.75754595, 0.45163566, 0.48747894],
      dtype=float32)

In [16]:
range(len(predictions))

range(0, 10)

In [17]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions))]

In [18]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['date']
)

In [19]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions
#result= result.sort_values(by=['timestamp'])

In [20]:
result

,timestamp,open_solana
0,1735998569,0.435249
1,1736084969,0.524755
2,1736171369,0.431021
3,1736257769,0.341056
4,1736344169,0.370243
5,1736430569,0.541414
6,1736516969,0.524726
7,1736603369,0.757546
8,1736689769,0.451636
9,1736776169,0.487479


In [21]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result.drop(columns=["timestamp"], inplace=True)

result

,open_solana,date
0,0.435249,2025-01-04
1,0.524755,2025-01-05
2,0.431021,2025-01-06
3,0.341056,2025-01-07
4,0.370243,2025-01-08
5,0.541414,2025-01-09
6,0.524726,2025-01-10
7,0.757546,2025-01-11
8,0.451636,2025-01-12
9,0.487479,2025-01-13


In [22]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [23]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    #overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_predictions_1_offline_fg_materialization/executions


(Job('solana_predictions_1_offline_fg_materialization', 'SPARK'), None)